In [1]:
import keras

Using TensorFlow backend.


In [2]:
from Models.functions.plot import plot_history, full_multiclass_report

SyntaxError: positional argument follows keyword argument (plot.py, line 275)

In [ ]:
from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.models import Model, Sequential
#from Models.functions.preprocessing import clean
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
import itertools
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def create_model(filters = [100], kernel_size = [50], strides = [100], 
                 dropout_rate = [0.5], pool_size = [5], max_len = 1000):

    model = Sequential()

    # conv 1
    model.add(Conv1D(filters = filters[0], 
                     kernel_size = kernel_size[0],
                     strides = strides[0], 
                     activation = 'relu', 
                     input_shape = (max_len, 1) ))

    # pooling layer 1
    model.add(MaxPooling1D(pool_size = pool_size[0], strides = 1))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(units = 100, activation = 'relu'))
    model.add(Dense(units = 2, activation = 'softmax'))

    model.compile(optimizer = 'adadelta', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
import pandas as pd
result = pd.DataFrame()

def get_results(model):

    config = model.get_config()

    row = {}

    conv_layers = np.sum([1 if i['class_name'] == "Conv1D" else 0 for i in config])
    pooling_layers = np.sum([1 if i['class_name'] == "MaxPooling1D" else 0 for i in config])

    row.update({ '_accuracy': accuracy_score(y_espected, y_predicted) })
    row.update({ '_f1-score': f1_score(y_espected, y_predicted,average='weighted')})
    row.update({ 'conv_layers': conv_layers })
    row.update({ 'pooling_layers': pooling_layers })

    _, _, fscore, support = precision_recall_fscore_support(y_espected, y_predicted)

    [row.update({'_fscore_class_'+str(i[0]): i[1]}) for i in enumerate(fscore)]
    [row.update({'_support_class_'+str(i[0]): i[1]}) for i in enumerate(support)]

    idx = 1
    for i in config:
        if i['class_name'] == "Conv1D":
            j = str(idx)
            row.update({
                'filters_'+j: i['config']['filters'],
                'strides_'+j: i['config']['strides'],
                'kernel_size_'+j: i['config']['kernel_size'],
                'activation_'+j: i['config']['activation']
            })
        pass
    return row

In [ ]:
def labelEncoder(y):
    le = preprocessing.LabelEncoder()
    le.fit(y)
    return (le.transform(y), len(le.classes_), list(le.classes_))

from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
from Models.functions.datasets import loadTrainTest
X_train, X_test, y_train, y_test = loadTrainTest("gender", "b5post", "/home/rafael/GDrive/Data/Dataframe/")

y_train, n, _ = labelEncoder(y_train)
y_test, n, classes_names = labelEncoder(y_test)

X = np.concatenate([X_train, X_test])

vect = TfidfVectorizer(max_features=1000)

vect.fit(X)

X.shape, X_train.shape, X_test.shape

In [ ]:
X_train_tfidf = vect.transform(X_train).toarray()
X_test_tfidf = vect.transform(X_test).toarray()

X_train_tfidf.shape, X_test_tfidf.shape

In [ ]:
max_length = np.max([len(x.split(" ")) for x in X_train])
mean_length = np.mean([len(x.split(" ")) for x in X_train])
mediam_length = np.median([len(x.split(" ")) for x in X_train])

print(max_length, mean_length, mediam_length, int(mediam_length), int(mediam_length) == 2255)

### Normalizer

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import collections, numpy

# Synthetic Minority Oversampling Technique (SMOTE)
def oversampling(X, y):
    X_resampled, y_resampled = SMOTE().fit_resample(X, y)
    return X_resampled, y_resampled

X_resampled, y_resampled = oversampling(X_train_tfidf, y_train)

#scaler = StandardScaler().fit(X)
#m = scaler.transform(X)
#m = matrix2

collections.Counter(y_train), collections.Counter(y_resampled)

### Split train and test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


from keras.utils import to_categorical
#y_train = to_categorical(y_train, 2)
#y_test = to_categorical(y_test, 2)
print(len(y_train), len(x_train))
print(len(y_test), len(x_test))
#print()
#print(y_test)

x_test.shape, x_train[1].shape

In [ ]:
xid, yid = 10, 222
plt.scatter(x_train[:,xid], x_train[:,yid],c=y_train)
#plt.ylabel(classes_names[0])
#plt.ylabel(classes_names[1])

In [ ]:
from time import time

CNN = KerasClassifier(build_fn=create_model, 
                epochs=1,
                #max_len=x_train[1],
                #batch_size=BATCH_SIZE,
                verbose=0,
                validation_split=0.1
                )

params_grid = dict(
        filters = [[40]],
        kernel_size = [[50]],
        strides = [[100]],
        dropout_rate = [[0.5]],
        pool_size = [[5]],
        epochs = [1]        
        #batch_size = 100
)                               

grid_search = GridSearchCV(CNN, 
                           params_grid, 
                           scoring='accuracy', cv=3, 
                           return_train_score=True
                           )

#print('best params', grid_search.best_params_)


print("Performing grid search...")    
t0 = time()

#histories = []
grid_results = grid_search.fit(x_train,y_train)

print("done in %0.2fs and %0.1fmin" % ((time() - t0), ((time() - t0) / 60) ))
print()

print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
display(pd.DataFrame(grid_search.cv_results_))

In [ ]:
params = best_parameters

## create the model with the best params found
model = create_model(filters=params['filters'],
                     kernel_size=params['kernel_size'],
                     strides=params['strides'],
                     dropout_rate=params['dropout_rate'],
                     pool_size=params['pool_size']
                    )

## Then train it and display the results
history = model.fit(x_train,
                    y_train,
                    epochs=5,#params['epochs'],
                    #batch_size=params['batch_size'],
                    verbose = 0)

model.summary()

plot_history(history)


full_multiclass_report(model,
                       x_test,
                       y_test,
                       classes=classes_names
                      )
                       #batch_size=32,
                       #binary= )

In [ ]:
from Models.functions import plot